In [11]:
import torch
import json

In [12]:
with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/MaxMinData.json', 'r') as file:
    data = json.load(file)
PDmax = data['PDmax']
PDmin = data['PDmin']
Tmax = data['Tmax']
Tmin = data['Tmin']
ACCmax = data['ACCmax']
ACCmin = data['ACCmin']

In [13]:
PDmax, PDmin, Tmax, Tmin, ACCmax, ACCmin

(10.831339999999908, 0.0, 30.5, 25.0, 32.0, 21.0)

In [14]:
def temp_normalize(T):
    return ((T - Tmin)/(Tmax - Tmin))
        
def PD_normalize(PD):    
    return ((PD - PDmin) / (PDmax - PDmin))

def ACC_normalize(ACC):
    return ((ACC - ACCmin) / (ACCmax - ACCmin))

def temp_denormalize(T):
    return ((Tmax - Tmin) * T + Tmin)
        
def PD_denormalize(PD):    
    return ((PDmax - PDmin) * PD + PDmin)

def ACC_denormalize(ACC):
    return ((ACCmax - ACCmin) * ACC + ACCmin)

In [15]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/PD_Model.pkl'
loaded_model = torch.load(path)
PD_model = loaded_model['model'].to(device)
PD_model.eval()

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Temp_Model.pkl'
loaded_model = torch.load(path)
Temp_model = loaded_model['model'].to(device)
Temp_model.eval()

LSTM(
  (lstm): LSTM(9, 64, num_layers=2, batch_first=True)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [16]:
ori_data = torch.load('/home/remote_user/sharefile/AIoT_code/time_series_model/data/dataset.pt')
ori_data.shape

torch.Size([9526, 30, 9])

In [17]:
ori_data[100][0][0]

tensor(0.1818)

In [18]:
temp_denormalize(ori_data[100][0][0])

tensor(26.)

In [19]:
for i in range(0, 30, 10):
    print('室內溫度 :', round(temp_denormalize(ori_data[i][-1][0]).item(), 4))
    for j in [21,22,23,24,25,26,27,32]:
        jj = ACC_denormalize(j)
        ori_data[i][-3][2] = jj
        ori_data[i][-2][2] = jj
        ori_data[i][-1][2] = jj
        res = Temp_model(ori_data[i].unsqueeze(0).to(device))
        res = temp_denormalize(res.item())
        print(j,"度:", round(res,4))
    print('='*20)
        

室內溫度 : 27.14
21 度: 29.4128
22 度: 29.4164
23 度: 29.4196
24 度: 29.4218
25 度: 29.4238
26 度: 29.4262
27 度: 29.4275
32 度: 29.4332
室內溫度 : 26.74
21 度: 29.3024
22 度: 29.3083
23 度: 29.3134
24 度: 29.3171
25 度: 29.3209
26 度: 29.3241
27 度: 29.3264
32 度: 29.3353
室內溫度 : 26.36
21 度: 29.1848
22 度: 29.1923
23 度: 29.1986
24 度: 29.2039
25 度: 29.2087
26 度: 29.2124
27 度: 29.2151
32 度: 29.2276


In [20]:
import torch

# 創建二維 Tensor
tensor = torch.tensor([[1, 2, 3, 4],
                       [5, 6, 7, 8],
                       [9, 10, 11, 12]])

# 提取範圍 [1][0] 到 [1][3] 的所有數據
extracted_data = tensor[1, 0:4]

# 打印結果
print(extracted_data)


tensor([5, 6, 7, 8])
